# How-to: Predict and update URLs for OPD data

## Environment pre-reqs
To install all required packages from `requirements.txt` in this folder, run:

In [ ]:
pip install -r requirements.txt

Or, if you are using conda, you can create a new environment and install the requirements with:

In [ ]:
conda create -n opd-env python=3.11
conda activate opd-env
pip install -r requirements.txt

## Prediction functions

In [6]:
# import local script
import prediction_funcs

## Example: Try to find a valid URL for a new year

**Additional spreadsheet fields:**
"Description", "source_url", "readme", "date_field",
"agency_field", "min_version", "query"

The `try_urls_years` function auto-fills the spreadsheet's "URL", "Year","last_coverage_check", "coverage_start", and "coverage_end".

In [7]:
url = "https://services5.arcgis.com/54falWtcpty3V47Z/arcgis/rest/services/Sacramento_Call_for_Service_Data_2024/FeatureServer/0"
spreadsheet_fields = { #required fields
    "State": "CA",
    "SourceName": "Example PD",
    "Agency": "EXPD",
    "AgencyFull": "Example Police Department",
    "TableType": "Arrests",
    "DataType": "ArcGIS"
    # below required if "DataType" = "Carto" or "Ckan"
    # , dataset_id": "example-dataset-id" 
}

prediction_funcs.try_url_years(url, n_years=1, verbose=True, spreadsheet_fields=spreadsheet_fields)

Valid URL found: https://services5.arcgis.com/54falWtcpty3V47Z/arcgis/rest/services/Sacramento_Call_for_Service_Data_2025/FeatureServer/0. Adding to OPD_Source_table.
Added to OPD_Source_table: https://services5.arcgis.com/54falWtcpty3V47Z/arcgis/rest/services/Sacramento_Call_for_Service_Data_2025/FeatureServer/0


## Example: Automatically update all outdated sources

In [3]:
prediction_funcs.auto_update_sources(
    # outdated_days=30 # unspecified defaults to last year
    # ,
    # verbose=True
)